In [1]:
import numpy as np
import sklearn as sk
import pandas as pd
import keras as ks
import spotipy

from keras.models import Sequential
from keras.layers import Dense, Dropout

from sklearn import preprocessing
from sklearn import model_selection

Using TensorFlow backend.


# Now that we have CSV files written, lets clean and process that data for use in training a model


In [45]:
df = pd.read_csv('data.csv')
df2 = pd.read_csv('data2.csv')
labels = ks.utils.np_utils.to_categorical([1 for _ in range(df.shape[0])] + [0 for _ in range(df2.shape[0])])
df3 = df.append(df2)
data3 = df3.values
data3 = np.delete(data3, 0, axis=1)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


Here we'll process our data before we train a model - we have to normalize and split into test and train sets

In [26]:
xTrain, xTest, yTrain, yTest = sk.model_selection.train_test_split(data3, labels, test_size=.25)

In [27]:
scaler = preprocessing.StandardScaler().fit(xTrain)
trainScaled = scaler.transform(xTrain)
testScaled = scaler.transform(xTest)

# MODEL TIME WOOOOO

In [72]:
model = Sequential()
model.add(Dense(32, input_dim=11, activation='sigmoid'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [73]:
model.fit(trainScaled, yTrain, epochs=1000, batch_size=64)

Epoch 1/1000
15261/15261 [==============================] - 1s 72us/step - loss: 0.5678 - acc: 0.7126
Epoch 2/1000
15261/15261 [==============================] - 0s 24us/step - loss: 0.5045 - acc: 0.7585
Epoch 3/1000
15261/15261 [==============================] - 0s 25us/step - loss: 0.4956 - acc: 0.7616
Epoch 4/1000
15261/15261 [==============================] - 0s 25us/step - loss: 0.4905 - acc: 0.7644
Epoch 5/1000
15261/15261 [==============================] - 0s 25us/step - loss: 0.4850 - acc: 0.7675
Epoch 6/1000
15261/15261 [==============================] - 0s 26us/step - loss: 0.4811 - acc: 0.7696
Epoch 7/1000
15261/15261 [==============================] - 0s 25us/step - loss: 0.4778 - acc: 0.7698
Epoch 8/1000
15261/15261 [==============================] - 0s 26us/step - loss: 0.4750 - acc: 0.7723
Epoch 9/1000
15261/15261 [==============================] - 0s 26us/step - loss: 0.4731 - acc: 0.7734
Epoch 10/1000
15261/15261 [==============================] - 0s 25us/step - loss: 

15261/15261 [==============================] - 0s 21us/step - loss: 0.4253 - acc: 0.7992
Epoch 81/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.4249 - acc: 0.8002
Epoch 82/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.4246 - acc: 0.7998
Epoch 83/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.4243 - acc: 0.8013
Epoch 84/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.4236 - acc: 0.8017
Epoch 85/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.4235 - acc: 0.8026
Epoch 86/1000
15261/15261 [==============================] - 0s 24us/step - loss: 0.4233 - acc: 0.8026
Epoch 87/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.4227 - acc: 0.8021
Epoch 88/1000
15261/15261 [==============================] - 0s 23us/step - loss: 0.4227 - acc: 0.8015
Epoch 89/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.422

15261/15261 [==============================] - 1s 35us/step - loss: 0.4066 - acc: 0.8084
Epoch 159/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.4055 - acc: 0.8123: 0s - loss: 0.4087 - acc: 0
Epoch 160/1000
15261/15261 [==============================] - 1s 35us/step - loss: 0.4053 - acc: 0.8123
Epoch 161/1000
15261/15261 [==============================] - 1s 52us/step - loss: 0.4054 - acc: 0.8099
Epoch 162/1000
15261/15261 [==============================] - 1s 60us/step - loss: 0.4057 - acc: 0.8101
Epoch 163/1000
15261/15261 [==============================] - 1s 97us/step - loss: 0.4055 - acc: 0.8108
Epoch 164/1000
15261/15261 [==============================] - 1s 42us/step - loss: 0.4045 - acc: 0.8128
Epoch 165/1000
15261/15261 [==============================] - 1s 59us/step - loss: 0.4048 - acc: 0.8101
Epoch 166/1000
15261/15261 [==============================] - 1s 42us/step - loss: 0.4049 - acc: 0.8103: 0s - loss: 0.4146 - 
Epoch 167/1000
15261/15261 [=

15261/15261 [==============================] - 0s 21us/step - loss: 0.3959 - acc: 0.8144
Epoch 235/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3964 - acc: 0.8142
Epoch 236/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3963 - acc: 0.8145
Epoch 237/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3959 - acc: 0.8141
Epoch 238/1000
15261/15261 [==============================] - 0s 25us/step - loss: 0.3955 - acc: 0.8170
Epoch 239/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3953 - acc: 0.8157
Epoch 240/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3956 - acc: 0.8172
Epoch 241/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3954 - acc: 0.8138
Epoch 242/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3955 - acc: 0.8181
Epoch 243/1000
15261/15261 [==============================] - 0s 22us/step - lo

15261/15261 [==============================] - 0s 22us/step - loss: 0.3896 - acc: 0.8196
Epoch 313/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3890 - acc: 0.8196
Epoch 314/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3888 - acc: 0.8207
Epoch 315/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3890 - acc: 0.8193
Epoch 316/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3891 - acc: 0.8196
Epoch 317/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3886 - acc: 0.8216
Epoch 318/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3883 - acc: 0.8213
Epoch 319/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3885 - acc: 0.8196
Epoch 320/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3888 - acc: 0.8207
Epoch 321/1000
15261/15261 [==============================] - 0s 23us/step - lo

...

15261/15261 [==============================] - 0s 23us/step - loss: 0.3742 - acc: 0.8290
Epoch 624/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3743 - acc: 0.8283
Epoch 625/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3744 - acc: 0.8270
Epoch 626/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3742 - acc: 0.8281
Epoch 627/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3746 - acc: 0.8301
Epoch 628/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3737 - acc: 0.8305
Epoch 629/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3748 - acc: 0.8291
Epoch 630/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3736 - acc: 0.8292
Epoch 631/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3741 - acc: 0.8300
Epoch 632/1000
15261/15261 [==============================] - 0s 24us/step - lo

15261/15261 [==============================] - 0s 22us/step - loss: 0.3724 - acc: 0.8300
Epoch 702/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3720 - acc: 0.8309
Epoch 703/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3727 - acc: 0.8313
Epoch 704/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3727 - acc: 0.8296
Epoch 705/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3723 - acc: 0.8307
Epoch 706/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3721 - acc: 0.8303
Epoch 707/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3722 - acc: 0.8302
Epoch 708/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3714 - acc: 0.8326
Epoch 709/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3723 - acc: 0.8312
Epoch 710/1000
15261/15261 [==============================] - 0s 21us/step - lo

15261/15261 [==============================] - 0s 24us/step - loss: 0.3712 - acc: 0.8313
Epoch 780/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3708 - acc: 0.8318
Epoch 781/1000
15261/15261 [==============================] - 0s 27us/step - loss: 0.3699 - acc: 0.8321
Epoch 782/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3701 - acc: 0.8331
Epoch 783/1000
15261/15261 [==============================] - 0s 23us/step - loss: 0.3703 - acc: 0.8306
Epoch 784/1000
15261/15261 [==============================] - 0s 24us/step - loss: 0.3703 - acc: 0.8311
Epoch 785/1000
15261/15261 [==============================] - 0s 22us/step - loss: 0.3710 - acc: 0.8313
Epoch 786/1000
15261/15261 [==============================] - 0s 23us/step - loss: 0.3703 - acc: 0.8320
Epoch 787/1000
15261/15261 [==============================] - 0s 21us/step - loss: 0.3706 - acc: 0.8311
Epoch 788/1000
15261/15261 [==============================] - 0s 21us/step - lo

15261/15261 [==============================] - 1s 60us/step - loss: 0.3687 - acc: 0.8320
Epoch 858/1000
15261/15261 [==============================] - 1s 83us/step - loss: 0.3686 - acc: 0.8317
Epoch 859/1000
15261/15261 [==============================] - 1s 59us/step - loss: 0.3687 - acc: 0.8328
Epoch 860/1000
15261/15261 [==============================] - 1s 59us/step - loss: 0.3692 - acc: 0.8322
Epoch 861/1000
15261/15261 [==============================] - 1s 64us/step - loss: 0.3688 - acc: 0.8305
Epoch 862/1000
15261/15261 [==============================] - 1s 59us/step - loss: 0.3690 - acc: 0.8303
Epoch 863/1000
15261/15261 [==============================] - 1s 80us/step - loss: 0.3684 - acc: 0.8303
Epoch 864/1000
15261/15261 [==============================] - 1s 89us/step - loss: 0.3692 - acc: 0.8313
Epoch 865/1000
15261/15261 [==============================] - 1s 90us/step - loss: 0.3681 - acc: 0.8318
Epoch 866/1000
15261/15261 [==============================] - 1s 78us/step - lo

15261/15261 [==============================] - 1s 87us/step - loss: 0.3675 - acc: 0.8324
Epoch 936/1000
15261/15261 [==============================] - 1s 75us/step - loss: 0.3670 - acc: 0.8317
Epoch 937/1000
15261/15261 [==============================] - 1s 62us/step - loss: 0.3677 - acc: 0.8323
Epoch 938/1000
15261/15261 [==============================] - 1s 63us/step - loss: 0.3669 - acc: 0.8320
Epoch 939/1000
15261/15261 [==============================] - 1s 60us/step - loss: 0.3680 - acc: 0.8318
Epoch 940/1000
15261/15261 [==============================] - 1s 63us/step - loss: 0.3679 - acc: 0.8334
Epoch 941/1000
15261/15261 [==============================] - 1s 64us/step - loss: 0.3676 - acc: 0.8334
Epoch 942/1000
15261/15261 [==============================] - 1s 60us/step - loss: 0.3674 - acc: 0.8324
Epoch 943/1000
15261/15261 [==============================] - 1s 61us/step - loss: 0.3679 - acc: 0.8312
Epoch 944/1000
15261/15261 [==============================] - 1s 61us/step - lo

In [74]:
model.evaluate(testScaled, yTest)


5088/5088 [==============================] - 1s 280us/step


[0.4664974795572413, 0.7929441823899371]